In [0]:
%sql
create schema migration

In [0]:
%sql
use migration;

## auto loader

In [0]:
%sql

desc table migration.bronze_table

col_name,data_type,comment
pk,int,null
name,string,null
hobby,string,null


In [0]:
%sql
set spark.databricks.delta.properties.defaults.autoOptimize.optimizeWrite = true;
--쓰기중에 자동 압축은 델타 테이블 파티션 내의 작은 파일을 결합하여 작은 파일 문제를 자동으로 줄입니다

key,value
spark.databricks.delta.properties.defaults.autoOptimize.optimizeWrite,true


In [0]:
%sql
set spark.databricks.delta.properties.defaults.autoOptimize.autoCompact = true;

key,value
spark.databricks.delta.properties.defaults.autoOptimize.autoCompact,true


In [0]:
%sql
set spark.databricks.delta.properties.defaults.enableChangeDataFeed = true;
-- 변경 데이터 피드를 활성화

key,value
spark.databricks.delta.properties.defaults.enableChangeDataFeed,true


In [0]:
schema = "Op string, pk int, name string, hobby string"

auto_df = spark.readStream \
            .format("cloudFiles") \
            .option("cloudFiles.format", "parquet") \
            .schema(schema) \
            .load("/mnt/jenny-migration/jennydb/raw/") \
            .writeStream \
            .format("delta") \
            .outputMode("append") \
            .trigger(once=True) \
            .option("checkpointLocation", "/mnt/checkpoints/cdc_231204") \
            .table("migration.cdc_df")

def merge(df,i):
    df._jdf.sparkSession().sql("""
    merge into migration.bronze_df as b
    using migration.cdc_df as c
    on b.pk = c.pk
    when matched and c.Op = 'D' Then delete
    when matched and c.Op = 'U' then update set *
    when not matched and c.Op = 'I' then insert *""")

cdc_query = spark.readStream \
            .table('migration.bronze_df') \
            .writeStream \
            .foreachBatch(merge) \
            .option("checkpointLocation", "/mnt/checkpoints/bronze_df") \
            .option("ignoreChanges", "true") \
            .trigger(once=True) \
            .start()

cdc_query

Out[72]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f80fc59d5b0>

In [0]:
%sql
select * from bronze_df order by pk asc;

pk,name,hobby
1,dsf,bts
2,fds,cx
3,vxc,hfg
4,ewr,hg
5,iuy,rty
6,hgf,jh
7,khj,bvc
8,dfs,nmb
9,bnv,i
10,ope,we
